In [1]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
#maybe not useful:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [2]:
from sklearn.svm import SVC

In [3]:
from sklearn.model_selection import GridSearchCV

In [2]:
"""
x_train = pd.read_csv("../data/x_train.csv", encoding = "ISO-8859-1")
y_train = pd.read_csv("../data/y_train.csv", encoding = "ISO-8859-1")

x_val = pd.read_csv("../data/x_val.csv", encoding = "ISO-8859-1")
y_val = pd.read_csv("../data/y_val.csv", encoding = "ISO-8859-1")

In [5]:
df= pd.read_csv('../data/final.csv')

In [6]:
df[df.doc.isnull()]

,Unnamed: 0,level_0,index,text,directed_at,emotion,target,brand,doc
6,6,6,6,NaN,NaN,No emotion toward brand or product,0,NaN,NaN


In [7]:
df.drop(index=6, axis=0, inplace=True)

In [7]:
df.shape

(8936, 9)

In [8]:
target = df["target"]
features = df['doc']
features

0       .@wesley83 3 G iPhone 3 hrs tweet rise_austin ...
1       @jessedee know @fludapp Awesome iPad iPhone ap...
2                       @swonderlin wait iPad 2 sale SXSW
3       @sxsw hope year festival crashy year iPhone ap...
4       @sxtxstate great stuff Fri SXSW Marissa Mayer ...
                              ...                        
8932                                       Ipad SXSW link
8933    wave buzz rt @mention interrupt regularly sche...
8934    Google Zeiger physician report potential ae FD...
8935    Verizon iPhone customer complain time fall hou...
8936       RT @mention Google Tests check offer SXSW link
Name: doc, Length: 8936, dtype: object

In [7]:
target.shape

(8936,)

In [28]:
features.shape

(8936,)

### Train Test Split

In [9]:
x_full, x_test, y_full, y_test = train_test_split(features, target, test_size=.2)
x_train, x_val, y_train, y_val = train_test_split(x_full, y_full, test_size=.2)

## Feature Engineering


In [10]:
tf = TfidfVectorizer()

In [11]:
tf_train = tf.fit_transform(x_train)
tf_val = tf.transform(x_val)
train_df=pd.DataFrame(tf_train.todense(), columns=tf.get_feature_names())
val_df=pd.DataFrame(tf_val.todense(), columns=tf.get_feature_names())

### Resampling

### Naive Bayes

In [12]:
gnb = GaussianNB()

In [13]:
gnb.fit(train_df, y_train)
y_pred = gnb.predict(val_df)

In [14]:
accuracy_score(y_pred, y_val)

0.46153846153846156

In [15]:
confusion_matrix(y_val,y_pred)

array([[331, 164, 360],
       [ 35,  31,  24],
       [114,  73, 298]])

In [ ]:
gnb.

### Multinomial Bayes

In [26]:
mnb = MultinomialNB(alpha=0.4)
mnb.fit(train_df, y_train)
y_pred = mnb.predict(val_df)
f1_score(y_pred, y_val, average='macro')

0.4540002666202856

In [32]:
from sklearn.metrics import f1_score, make_scorer

f1 = make_scorer(f1_score , average='macro')

In [34]:
params = [{'alpha':[x/20 for x in list(range(0, 20))]}]
gridsearch = GridSearchCV( MultinomialNB(), params, scoring=f1, cv=5, verbose=1, n_jobs=-1)

In [35]:
gridsearch.fit(train_df,y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   32.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   58.7s finished


GridSearchCV(cv=5, estimator=MultinomialNB(), n_jobs=-1,
             param_grid=[{'alpha': [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35,
                                    0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75,
                                    0.8, 0.85, 0.9, 0.95]}],
             scoring=make_scorer(f1_score, average=macro), verbose=1)

In [38]:
gridsearch.best_params_

{'alpha': 0.05}

In [ ]:
gridsearch.mo

In [40]:
mnb = MultinomialNB(gridsearch.best_params_['alpha'])
mnb.fit(train_df, y_train)
y_pred = mnb.predict(val_df)
f1_score(y_pred, y_val, average='macro')
accuracy_score(y_pred, y_val)

/Users/antonhaugen/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass alpha=0.05 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


0.6524475524475525

In [41]:
confusion_matrix(y_val,y_pred)

array([[684,  18, 153],
       [ 53,  14,  23],
       [247,   3, 235]])

### Support Vector Classification

In [ ]:
svc= SVC(C=10, class_weight='balanced', gamma='scale', kernel='rbf')
svc.fit(train_df,  y_train)

In [95]:
y_pred = svc.predict(val_df)
print(accuracy_score(y_pred, y_val))

0.8679577464788732 0.9259624876604146


In [97]:
confusion_matrix(y_val,y_pred)

array([[ 24,  63],
       [ 12, 469]])